# Santander 2025 - Ciência de Dados (ETL com Python)

#Contexto
Você é um cientista de dados em um mundo de fantasia e recebeu a tarefa de montivar seus heróis de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de motivacionais personalizadas que serão entregues a cada herói.

In [1]:
# Utilizei minha própria URL

superhero_api_url = 'https://uninquisitive-axel-undecorous.ngrok-free.dev'


## **E**xtract

Foi extraída a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, foi feita uma requisição GET para obter os dados do usuário correspondente.

In [2]:
import pandas as pd

df = pd.read_csv('superhero.csv')
user_ids = df['User ID'].tolist()
print(user_ids)

[1, 2, 3, 4, 5]


In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{superhero_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

## **T**ransform

Foi utilizada a API do Hugging Face para gerar uma mensagem motivacional personalizada para cada usuário (herói).

In [ ]:
!pip install openai

In [ ]:

# Para gerar uma API Key:
# 1. Crie uma conta no Hugging Face
# 2. "Access Token" na seção do seu perfil
# 3. Clique em "Create New Token"
# Link direto: https://huggingface.co/settings/tokens


import requests

HF_API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
HF_TOKEN = "Utilize seu Token aqui"

headers = {
    "Authorization": f"Bearer {HF_TOKEN}"
}


def gerar_mensagem_ia(prompt):
    payload = {
        "inputs": {
            "text": prompt
        }
    }

    response = requests.post(
        HF_API_URL,
        headers=headers,
        json=payload
    )

    return response.json()



In [31]:
import os
from openai import OpenAI

# Inicializa o cliente OpenAI usando o HF_TOKEN diretamente
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN, # Usa a variável HF_TOKEN diretamente
)

def criar_prompt(user):
    return f"""
    Crie uma mensagem curta, criativa e personalizada para o super-herói {user['name']}.
    Características do herói: {user['news']}.
    A mensagem deve ser inspiradora e motivacional.
    """

def generate_ai_news(user):
  # Usa o cliente inicializado globalmente e o prompt personalizado
  completion = client.chat.completions.create(
      model="moonshotai/Kimi-K2-Instruct-0905", # Usando o modelo especificado
      messages=[
          {"role": "user", "content": criar_prompt(user)}
      ],
  )
  return completion.choices[0].message.content.strip()

for user in users:
  news = generate_ai_news(user)
  print(f"Notícia gerada para {user['name']}: {news}")
  user['news'].append({
      "description": news
  })


Notícia gerada para Iron Man: Tony, o verdadeiro reactor és tu: um coração que pulsa ideias mais rápido que luz. Quando a armadura se fecha, fecha-se também a dúvida — e abre-se o futuro.
Notícia gerada para Captain America: “Steve, erga o escudo: cada batida do seu coração é um tambor que convoca o mundo a marchar na cadência da coragem. Avance — onde você pisar, o medo vira ponte e a justiça vira caminho.”
Notícia gerada para Thor: “Thor, faça Mjölnir girar uma vez — não no céu, mas dentro do meu peito. Transforma o medo em trovão e o meu coração em Asgard: pequeno, mas indestrutível. Que cada pulso meu seja um raio que proteja o meu mundo. Skål ao deus que carrego!”
Notícia gerada para Black Widow: “Natasha, o vermelho que um dia te aprisionava agora te impulsiona: cada cicatriz virou estrela, cada segredo virou combustível. Voa, viúva – o céu não é mais limite, é assinatura.”
Notícia gerada para Spider-Man: “Peter, pendura o uniforme no vento: cada teia é um verso inédito que só o 

## **L**oad

Por fim, foi atualizada a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [5]:
def update_user(user):
  response = requests.put(f"{superhero_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} atualizado? {success}!")

User Iron Man atualizado? True!
User Captain America atualizado? True!
User Thor atualizado? True!
User Black Widow atualizado? True!
User Spider-Man atualizado? True!
